[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nakamura196/ddc-tutorial2025/blob/main/notebooks/tutorial_03_palladio/tutorial_03_palladio.ipynb)

# 可視化3：地図による空間分析（Palladio）

**出版地の地理的分布を可視化する**

江戸後期の出版活動がどの都市で行われていたかを、地図上で可視化します。

## 使用ツール
- **Palladio**: http://hdlab.stanford.edu/palladio/
  - スタンフォード大学が開発したDH向け可視化ツール
  - Webブラウザで動作（インストール不要）
  - Map（地図）・Graph（ネットワーク）・List・Galleryの4つのビュー

## 目的
- 出版活動の地理的分布を把握する
- 江戸・京都・大阪の「三都」の出版規模を比較する

---
## 1. データの読み込み

In [ ]:
import pandas as pd

# Google Colabでファイルをアップロードする場合
try:
    from google.colab import files
    print("ndl_sru_for_rawgraphs.csv をアップロードしてください")
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
    df = pd.read_csv(filename)
except ImportError:
    # ローカル環境の場合
    df = pd.read_csv('../../data/ndl_sru_for_rawgraphs.csv')

print(f"データ件数: {len(df)}件")
df.head()

---
## 2. 出版地の分布を確認

In [ ]:
# 出版地の分布
print("=== 出版地の分布 ===")
print(df['location'].value_counts())

---
## 3. 座標情報の追加

地図上に表示するには、緯度（lat）と経度（lon）が必要です。
出版地名に対応する座標を追加します。

**座標データの出典**: [Wikidata](https://www.wikidata.org/)

In [ ]:
# 出版地の座標データ（Wikidataより取得）
# 出典: https://www.wikidata.org/
LOCATION_COORDS = {
    # 旧地名: (緯度, 経度, Wikidata ID, 現代の地名)
    '江戸': (35.683889, 139.774444, 'Q7473516', '東京'),
    '京都': (35.011611, 135.768111, 'Q34600', '京都市'),
    '大阪': (34.69375, 135.502111, 'Q35765', '大阪市'),
    '名古屋': (35.181389, 136.906389, 'Q11751', '名古屋市'),
    '金沢': (36.561056, 136.6565, 'Q191130', '金沢市'),
    '長崎': (32.749528, 129.879639, 'Q38234', '長崎市'),
}

# 座標を追加
def add_coordinates(row):
    location = row['location']
    if location in LOCATION_COORDS:
        lat, lon, wikidata_id, modern_name = LOCATION_COORDS[location]
        return pd.Series([lat, lon, wikidata_id])
    else:
        return pd.Series([None, None, None])

df[['lat', 'lon', 'wikidata_id']] = df.apply(add_coordinates, axis=1)

print("座標を追加しました（出典: Wikidata）")
print("\n=== 座標データ ===")
for loc, (lat, lon, wid, modern) in LOCATION_COORDS.items():
    print(f"  {loc}（{modern}）: {lat}, {lon} [wd:{wid}]")

---
## 4. Palladio用データの作成

座標のあるデータのみを抽出し、Palladio用のCSVを作成します。

In [ ]:
# 座標のあるデータのみ抽出
df_with_coords = df.dropna(subset=['lat', 'lon']).copy()

print(f"座標ありデータ: {len(df_with_coords)}件")
print(f"座標なしデータ: {len(df) - len(df_with_coords)}件（除外）")

# 出版地別の件数
print("\n=== 出版地別件数 ===")
print(df_with_coords['location'].value_counts())

In [ ]:
# Palladio用にカラムを整理
df_palladio = df_with_coords[['year', 'title', 'genre_major', 'location', 'lat', 'lon', 'wikidata_id', 'creator_normalized']].copy()

# Palladioの座標形式（lat,lon）を作成
df_palladio['coordinates'] = df_palladio['lat'].astype(str) + ',' + df_palladio['lon'].astype(str)

print(f"Palladio用データ: {len(df_palladio)}件")
df_palladio.head()

---
## 5. CSVファイルの出力

In [ ]:
# CSVファイルとして保存
csv_filename = 'ndl_for_palladio.csv'
df_palladio.to_csv(csv_filename, index=False, encoding='utf-8')

print(f"CSVファイルを作成しました: {csv_filename}")

In [ ]:
# Google Colabの場合、ファイルをダウンロード
try:
    from google.colab import files
    print("ファイルをダウンロードしています...")
    files.download(csv_filename)
except ImportError:
    print(f"ローカル環境: カレントディレクトリに保存しました")

---
## 6. Palladio での操作手順

### 6.1 データの読み込み
1. [Palladio](http://hdlab.stanford.edu/palladio/) にアクセス
2. 「**Start**」をクリック
3. ダウンロードした `ndl_for_palladio.csv` をドラッグ＆ドロップ
4. 「**Load**」をクリック

### 6.2 地図の作成（Map）
1. 上部タブから「**Map**」を選択
2. 「**New Layer**」をクリック
3. 設定:
   - **Places**: `coordinates` を選択
   - **Tooltip label**: `location` を選択
   - **Size points**: チェックを入れ、「**Count**」を選択（データ数で円のサイズが変わる）
4. 「**Add layer**」をクリック

### 6.3 フィルタリング（Facet）
1. 画面下部の「**Facet**」でフィルタ条件を追加
2. 「**Dimensions**」から `year` や `genre_major` を選択
3. 特定の年代やジャンルに絞り込んで表示

### 6.4 読み取りポイント

#### 三都の比較
- **江戸**（最大の円）: 幕府のお膝元、出版の中心地
- **大阪**: 商業出版の拠点
- **京都**: 伝統的な出版地

#### 地方都市
- 名古屋、金沢、長崎などの地方出版の存在を確認

---
## 7. 統計情報

In [ ]:
# 出版地×年代のクロス集計
print("=== 出版地×年代（10年単位）===")
df_palladio['decade'] = (df_palladio['year'] // 10) * 10
cross_tab = pd.crosstab(df_palladio['decade'], df_palladio['location'])
print(cross_tab)

In [ ]:
# 出版地×ジャンルのクロス集計
print("=== 出版地×ジャンル ===")
cross_tab2 = pd.crosstab(df_palladio['location'], df_palladio['genre_major'])
print(cross_tab2)

---
## まとめ

### 作成したファイル
| ファイル名 | 内容 | 用途 |
|-----------|------|------|
| `ndl_for_palladio.csv` | 座標付き出版データ | Palladioで地図表示 |

### 含まれるカラム
| カラム名 | 説明 |
|---------|------|
| year | 出版年 |
| title | 作品名 |
| genre_major | ジャンル大分類 |
| location | 出版地 |
| lat | 緯度 |
| lon | 経度 |
| wikidata_id | Wikidata ID（出典） |
| coordinates | 座標（lat,lon形式） |
| creator_normalized | 著者 |

### 座標データの出典
- **Wikidata** (https://www.wikidata.org/)

### 空間人文学（Spatial Humanities）への展開
- **Palladio**: 基本的な地図可視化
- **Flourish**: より高度なバブルチャート、円グラフ地図
- **QGIS**: 本格的なGIS分析